In [1]:
# set package parent path
import os
import sys

def get_parent_path(parent_folder_name: str = "src"):
    cur_path = os.path.abspath('')
    parent_path = cur_path[:cur_path.find(parent_folder_name) + len(parent_folder_name)]

    return parent_path   

parent_path = get_parent_path()
sys.path.append(parent_path)

In [2]:
import os

from huggingface_hub import HfApi, ModelFilter

from data.preprocessing.task_dataset.srl_dataset import SrlBioDataset
from data.preprocessing.task_dataset.task_dataset import (DatasetSize, Mode,
                                                          Task)
from models.explain.multitask.bilstm import BiLSTMExplainMultitask
from models.explain.multitask.linear import LinearExplainMultitask
from models.explain.singletask.bilstm import BiLSTMExplainSingleTask
from models.explain.singletask.linear import LinearExplainSingleTask
from models.explain.biLSTM.bilstm import BiLSTMExplainBiLSTM
# from models.explain.biLSTM.linear import LinearExplainLinear
from utils.measures.archivists.per_class import PerClassArchivist
from utils.measures.archivists.per_sentence import PerSentenceArchivist

from utils.measures.compute_methods.vector_representation import VectorRepresentation

from utils.measures.measure_computer import MultiProcessComputer

from utils.others import get_database
from utils.trainer import setup_before_train

setup_before_train(parent_path)
hf_api = HfApi(token=os.getenv("HUGGING_FACE_TOKEN"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /Users/Na/.cache/huggingface/token
Login successful


## Prepare model and configs

In [3]:
dener_model_name_list = hf_api.list_models()
dataset_size = DatasetSize.SMALL
dener_model_id_list = [model.modelId for model in dener_model_name_list]

1

In [4]:
# TODO: use your base model name
base_model_name = "MohametSena/bio_srl_bilstm_xs_v6-1"

for dener_model_name in dener_model_id_list:

    # TODO: use one of them
    from models.explain.multitask.bilstm import BiLSTMExplainMultitask
    from models.explain.singletask.bilstm import BiLSTMExplainSingleTask
    from models.explain.biLSTM.bilstm import BiLSTMExplainBiLSTM
    explain_model = BiLSTMExplainBiLSTM(
        dener_model=dener_model_name,
        base_model=base_model_name,
    )

    srl_dataset = SrlBioDataset(
        model_name=base_model_name, 
        label_file_name=f"{parent_path}/data/processed/{dataset_size}_bio_srl_label.csv"
    ) \
        .set_mode(Mode.COMPUTE_MEASURES) \
        .from_cache(f"{parent_path}/data/processed/{dataset_size}_bio_srl.csv")
    
    # TODO: use one of them
    from data.preprocessing.base_model_dataset.multitask_dataset import MultitaskBioDataset
    from data.preprocessing.base_model_dataset.singletask_dataset import SingleTaskBioDataset
    from data.preprocessing.base_model_dataset.bilstm_dataset import BiLSTMBioDataset
    dataset = BiLSTMBioDataset(
        # task_datasets={
        #     Task.SRL: srl_dataset,
        # },
        task_dataset=srl_dataset,
        train_test_valid_split=[0, 0, 1]
    )

    computer = MultiProcessComputer(explain_model, dataset, srl_dataset) \
        .set_num_proc(percent_cpu_to_use=0.2) \
        .set_pbar_color("#21D375")

    computer.add_compute_methods([])

    computer.add_archivists([
        PerClassArchivist(),
        PerSentenceArchivist(),
    ])

    computer.compute()

    # TODO: use one of them
    from utils.measures.savers.mongodb import MongoDBSaver
    from utils.measures.savers.json import JsonSaver

    computer.save(JsonSaver(f"{parent_path}/test_measures"))
    # computer.save(MongoDBSaver("test_measures"))

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


⬇️ Loading label vectorizer from /Users/Na/Project/BioModels/src/data/processed/sm_bio_srl_label.csv ...
MohametSena/1C_bilstm_DeNER_bilstm_SeparationLoss_xs_cell_line_v6-1-31


MultiProcessComputer (num_proc=1):  53%|█████▎    | 56/105 [07:41<06:57,  8.52s/it]

In [ ]:
from utils.shapley import Shapley

shapley = Shapley(["cell_line", "cell_type", "DNA", "protein", "RNA"])
shapley.set_base_model_type("singletask")
shapley.set_correlation_coefficient("spearman")

result = shapley.compute_correlation()
result